In [1]:
import pickle as pkl

path = '../data/y_pred.pkl'

with open(path, 'rb') as fid:
    y_pred = pkl.load(fid)

EOFError: Ran out of input

In [2]:
import rasterio
from rasterio.features import geometry_mask
from rasterio.windows import from_bounds
import numpy as np
from shapely.geometry import Polygon

def calculate_overlap_percentage(tif_path, polygon):
    """
    Calculates the % of the polygon area that overlaps with value=1 in the GeoTIFF.
    """
    with rasterio.open(tif_path) as src:
        
        # 1. Coordinate System Check
        # Ensure your polygon is in the same CRS as the GeoTIFF
        # If not, you must reproject the polygon before passing it here.
        
        # 2. Create a Window
        # This defines the slice of the raster array we need to read
        window = from_bounds(*polygon.bounds, transform=src.transform)
        
        # Handle edge case: If polygon is outside raster bounds completely
        if window.width == 0 or window.height == 0:
            return 0.0

        # 3. Read the Data
        # Read only the band (1) within that window
        # boundless=True handles cases where polygon extends slightly off the map
        raster_data = src.read(1, window=window, boundless=True, fill_value=0)
        
        # 4. Get the Transform for this specific window
        win_transform = src.window_transform(window)
        
        # 5. Rasterize the Polygon
        # Create a boolean mask where True = Inside Polygon
        # shape must match the read raster_data
        poly_mask = geometry_mask(
            [polygon],
            out_shape=raster_data.shape,
            transform=win_transform,
            invert=True,  # Invert=True makes the inside of the polygon True
            all_touched=False # False = center of pixel must be inside; True = any part touches
        )

        # 6. Calculate Overlap
        # Count pixels that are INSIDE the polygon
        total_poly_pixels = np.sum(poly_mask)

        if total_poly_pixels == 0:
            return 0.0

        # Count pixels that are INSIDE the polygon AND have value 1 in raster
        # We assume the binary mask uses 1 for "True". 
        overlap_pixels = np.sum((raster_data == 1) & poly_mask)

        # Calculate percentage
        percent_overlap = (overlap_pixels / total_poly_pixels) * 100
        
        return percent_overlap

# --- Example Usage ---

# Example: A small square
my_polygon = Polygon([(2949900,1280500),  (3227695,1178842), (3022261,1060594)])

# Path to your binary GeoTIFF
tif_file = "../predictions/inundation_predictions_2025-11-21_to_2026-01-11/flood_scenario_average.tif"

# Run calculation
try:
    result = calculate_overlap_percentage(tif_file, my_polygon)
    print(f"Overlap: {result:.2f}%")
except Exception as e:
    print(f"Error: {e}")

Overlap: 25.10%


In [17]:

import xarray as xr
import rioxarray
import numpy as np
import rasterio
from datetime import datetime

tif_files = {
             0.0: "../predictions/inundation_predictions_2025-11-21_to_2026-01-11/current_extent.tif",
             0.05: "../predictions/inundation_predictions_2025-11-21_to_2026-01-11/flood_scenario_best.tif",
             0.5: "../predictions/inundation_predictions_2025-11-21_to_2026-01-11/flood_scenario_average.tif",
             0.95: "../predictions/inundation_predictions_2025-11-21_to_2026-01-11/flood_scenario_worst.tif"}

rwds = []
for s, f in tif_files.items():
    raster = rioxarray.open_rasterio(f)
    rwd = xr.where(raster == 1.0, 0.5, -0.5).sel(band=1).rio.reproject('epsg:4326')
    rwds.append(rwd)
da = xr.concat(rwds, dim='s').assign_coords(s=list(tif_files.keys())).expand_dims({'t': [datetime.now()]})

ds = da.to_dataset(name='relative_water_depth').drop_vars('band')
#ds.to_zarr('../predctions/water_depth_tensor.zarr')
ds.to_netcdf('../predictions/water_depth_tensor.nc')

In [21]:
xr.load_dataset('../predictions/water_depth_tensor.nc')

<xarray.Dataset> Size: 41MB
Dimensions:               (t: 1, x: 1341, y: 957, s: 4)
Coordinates:
  * t                     (t) datetime64[ns] 8B 2025-12-01T12:42:44.101337
  * x                     (x) float64 11kB 23.43 23.44 23.45 ... 35.89 35.9
  * y                     (y) float64 8kB 12.3 12.29 12.28 ... 3.423 3.414 3.405
  * s                     (s) float64 32B 0.0 0.05 0.5 0.95
Data variables:
    spatial_ref           int64 8B 0
    relative_water_depth  (t, s, y, x) float64 41MB -0.5 -0.5 -0.5 ... -0.5 -0.5